In [58]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score

In [59]:
######## opening data files ########
def open_file(file):
    pixels = []
    curr = []
    f = open(file,'r')
    lines = f.readlines()
    counter = 0
    for line in lines:
        line = line[0:28]
        temp = []
        for ch in line:
            if ch == '#' or ch == '+':
                temp.append(1)
            else:
                temp.append(0)
        curr.append(temp)
        counter+=1
        if counter == 28:
            pixels.append(curr)
            curr = []
            counter = 0
    f.close()
    return pixels

In [60]:
######## opening label files ########
def open_label_file(label_file):
    y = []
    f = open(label_file,'r')
    lines = f.readlines()
    for line in lines:
        if line != '\n': 
            y.append(int(line))
    f.close()
    return y

In [61]:
######## importing data ########
train_x = open_file("trainingimages")
train_y = open_label_file("traininglabels")
val_x = open_file("validationimages")
val_y = open_label_file("validationlabels")
test_x = open_file("testimages")
test_y = open_label_file("testlabels")

In [62]:
c = 0
for i in train_x:
    for j in i:
        print(j)
    print('\n')
    c+=1
    if c == 3:
        break
    

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0

In [63]:
################### mapping pixels with their labeled digit, and storing the result in a dictionary ###################

def count_digit_appearance(train,labels):
    dictionary = {0: [], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}
    i = 0
#     print(train[i])
    for l in labels:
        dictionary[l].append(train[i])
        i+=1
    return dictionary

In [64]:
################### calculating prior probability P(C) ###################

def calculate_prior(train_labels):
    total = len(train_labels)
    prior = list(np.zeros(10))
    for y in train_labels:
        prior[y]+=1
    for i in range(len(prior)):
        prior[i] /= total
        prior[i] = math.log(prior[i])     
    return prior

In [65]:
################### calculating likelihood for each index in each one of the matrices, a helper function is called to perform smoothing ###################

def calc_likelihood(dict1,k):
    new_dict = {0: [], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]}
    for key in dict1:
        entry = dict1[key]
        ones = list(np.zeros((len(entry[0]),len(entry[0][0])),dtype=int))
        zeros = list(np.zeros((len(entry[0]),len(entry[0][0])),dtype=int))
        x_len = len(entry)
        for value in entry:
            for i in range(len(entry[0])):
                for j in range(len(entry[0][0])):         
                    if value[i][j] == 0:
                        zeros[i][j]+=1
                    else:
                        ones[i][j]+=value[i][j]
        new_dict[key] = ones,zeros,x_len
    new_dict = smoothing(new_dict,k)
    return new_dict
        
    
def smoothing(dict2, k):
    new_dict2 = ({0: [], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]})
    for key in dict2:
        ones = (dict2[key][0])
        ones = np.array(ones)
        ones = ones.astype(float)
        zeros = (dict2[key][1])
        zeros = np.array(zeros)
        zeros = zeros.astype(float)
        x_len = dict2[key][2]
        for i in range(len(ones)):
            for j in range(len(ones[0])):
                ones[i][j] += k
                zeros[i][j] +=k
        x_len += float(2*k)
        for i in range(len(ones)):
            for j in range(len(ones[0])):
                ones[i][j]/=x_len
                zeros[i][j]/=x_len
        new_dict2[key] = (ones,zeros,x_len)
    return new_dict2

In [66]:
def fit(train_pixels,train_labels,k):
    appearances = count_digit_appearance(train_x,train_labels)
    priors = calculate_prior(train_labels)
    likelihoods = calc_likelihood(appearances,k)
    return priors,likelihoods

In [67]:
def predict(priors,likelihoods,test_pixels):
    preds = []
    for pix in test_pixels:
        max_prob = float('-inf')
        for p in range(len(priors)):
            ones = likelihoods[p][0]
            zeros = likelihoods[p][1]
            prob = priors[p]
            for i in range(len(pix)):
                for j in range(len(pix[0])):
                    if pix[i][j] == 0.0:
                        prob+= math.log(zeros[i][j])
                    else:
                        prob+=math.log(ones[i][j])
            if prob > max_prob:
                max_prob = prob
                pred_digit = p
        preds.append(pred_digit)
    return preds

## Section A - Hyper Tuning The Model 

In [68]:
%%time
grid_search = {'k_value':[],'accuracy score': []}
k_list = [1,2,3,4,5]
for k in range(len(k_list)):
    print(k)
    val_params = fit(train_x,train_y,k+1)
    y_val_pred = predict(val_params[0],val_params[1],val_x)
    acc = round(accuracy_score(val_y, y_val_pred) *100 ,3)
    grid_search['k_value'].append(k+1)
    grid_search['accuracy score'].append(acc)

0
1
2
3
4
Wall time: 34.6 s


In [69]:
grid_search = pd.DataFrame(grid_search)
grid_search.sort_values('accuracy score',ascending=False).head()

,k_value,accuracy score
0,1,81.8
1,2,81.8
2,3,81.1
3,4,81.1
4,5,81.0


In [70]:
best_k = grid_search.sort_values('accuracy score',ascending=False).iloc[0][0]
best_k

1.0

## Section B - Testing With Tuned 'K'

In [71]:
params = fit(train_x,train_y,best_k)

In [72]:
predictions = predict(params[0],params[1],test_x)
acc_test = round(accuracy_score(test_y,predictions)*100,3)
print("Model Accuracy is: ",acc_test,"%")

Model Accuracy is:  77.1 %


## Section C - Testing On Training Labels

In [73]:
pred_training = predict(params[0],params[1],train_x)
acc_test = round(accuracy_score(train_y,pred_training)*100,3)
print("Model Accuracy is: ",acc_test,"%")

Model Accuracy is:  84.0 %


### Train accuracy is higher than test accuracy because the model was built and optimized based on the train features

<h2>PERCEPTRON

<h3> class and functions

In [74]:
def accuracy_calculator(y_true,y_pred):
    TNTF = 0
    labels_total = {}
    label_TNTF = {}
    all_accuracies = {}
    uniques = np.unique(y_true)
    
    for i in uniques:
        labels_total[i] = 0
        label_TNTF[i] = 0
        all_accuracies[i] = 0

    for i in range(len(y_true)):
        labels_total[y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            TNTF += 1
            label_TNTF[y_true[i]] += 1

    for i in uniques:
        all_accuracies[i] = label_TNTF[i]/labels_total[i] * 100
    
    return TNTF/len(y_true)*100, all_accuracies

#predicts the labels and chooses the perceptron with the highest accuracy
def predict(weights_list, images, labels, size):
    
    #bias
    images = np.hstack((np.ones((size, 1)), images))

    y_pred = np.dot(weights_list, images.T)
    y_pred = np.argmax(y_pred, axis=0).T

    avg_accuracy,all_accuracies = accuracy_calculator(labels,y_pred)
    
    return avg_accuracy,all_accuracies

#Calculate appropiate weights for each classifier
def train(train_images, train_labels, epoch):
    #adding bias
    train_images = np.hstack((np.ones((training_size, 1)), train_images))
    
    # initialize weights as all 0's
    weights_list = np.zeros((labels, train_images.shape[1]))
    train_labels = train_labels.reshape((training_size, 1))
    train_labels_copy = np.copy(train_labels)
    
    for ep in range(epoch):
        print
        for y_true,img in zip(train_labels,train_images):
            res = []
            for w in weights_list:
                res.append(np.dot(img,w))
            arr_res = np.array(res)
            y_pred = np.argmax(arr_res)
            if(y_true != y_pred):
                weights_list[y_pred] = weights_list[y_pred] - img
                weights_list[y_true] = weights_list[y_true] + img

    return weights_list

In [77]:
training_size = len(train_x)
testing_size = len(test_x)
val_size = len(val_x)
learning_rate = 1

train_images, train_labels = train_x,train_y
test_images, test_labels = test_x,test_y
val_images, val_labels = val_x,val_y

train_images = np.array(train_images)
train_labels = np.array(train_labels, dtype=np.int32)
    
val_images = np.array(val_images)
val_labels = np.array(val_labels, dtype=np.int32)

test_images = np.array(test_images)
test_labels = np.array(test_labels, dtype=np.int32)

train_images = train_images.reshape(train_images.shape[0],28*28)
test_images = test_images.reshape(test_images.shape[0],28*28)
val_images = val_images.reshape(val_images.shape[0],28*28)


In [78]:
train_images.shape

(5000, 784)

In [79]:
len(train_x)

5000

In [80]:
def init_data():
    # load data
    train_images, train_labels = train_x,train_y
    test_images, test_labels = test_x,test_y
    val_images, val_labels = val_x,val_y
    
    train_images = np.array(train_images)[:,:,:]
    train_labels = np.array(train_labels, dtype=np.int32)
    
    test_images = np.array(test_images)[:,:,:]
    test_labels = np.array(test_labels, dtype=np.int32)
    
    val_images = np.array(val_images)[:,:,:]
    val_labels = np.array(val_labels, dtype=np.int32)

    train_images = train_images.reshape(train_images.shape[0],28*28)
    test_images = test_images.reshape(test_images.shape[0],28*28)
    val_images = val_images.reshape(val_images.shape[0],28*28)

init_data()

<h3> Perceptron. A - EPOCHS = 3


In [81]:
epochs = 3
init_data()
print("Training")
weights_list = train(train_images, train_labels,epochs)
print("Done")

Training
Done


<h3> TRAIN DATA

In [82]:
%%time
accuracies = predict(weights_list, train_images, train_labels, training_size)
print('Average Training Accuracy:', accuracies[0], '%\n')
print('Accuracy for each perceptron:', accuracies[1], '%\n')

Average Training Accuracy: 85.2 %

Accuracy for each perceptron: {0: 92.48434237995825, 1: 94.84902309058614, 2: 97.3360655737705, 3: 78.90466531440163, 4: 90.65420560747664, 5: 80.87557603686636, 6: 91.61676646706587, 7: 95.81818181818181, 8: 36.14718614718615, 9: 86.86868686868688} %

Wall time: 31 ms


<h3> TEST DATA

In [83]:
%%time
test_acc = predict(weights_list, test_images, test_labels, len(test_labels))
print('Average Test Accuracy:', test_acc[0], '%\n')

print("{:<10} {:<10}".format('Perceptron','Accuracy'))
for k, v in test_acc[1].items():    
    print("{:<10} {:<10}".format(k, v))


Average Test Accuracy: 76.3 %

Perceptron Accuracy  
0          83.33333333333334
1          96.29629629629629
2          93.20388349514563
3          72.0      
4          80.37383177570094
5          65.21739130434783
6          80.21978021978022
7          81.13207547169812
8          28.155339805825243
9          82.0      
Wall time: 7.34 ms


<h3> Perceptron. B - EPOCHS -> [1,5]


In [84]:
%%time
epochs_train = {}
epochs_test = {}
init_data()
for ep in range(1,6):
    print('Current Epoch : ',ep)
    epochs = ep
    init_data()
    weights_list = train(train_images, train_labels,ep)
    epochs_train[ep] = predict(weights_list, train_images, train_labels, training_size)
    epochs_test[ep] = predict(weights_list, test_images, test_labels, testing_size) 

Current Epoch :  1
Current Epoch :  2
Current Epoch :  3
Current Epoch :  4
Current Epoch :  5
Wall time: 6.15 s


In [85]:
for i in range(1,6):
    print('ACCURACY FOR EP', i)
    print('   TRAIN ACCURACY : ',epochs_train[i][0],'%')
    print('   TEST ACCURACY : ',epochs_test[i][0],'%')

ACCURACY FOR EP 1
   TRAIN ACCURACY :  87.0 %
   TEST ACCURACY :  78.10000000000001 %
ACCURACY FOR EP 2
   TRAIN ACCURACY :  89.1 %
   TEST ACCURACY :  81.2 %
ACCURACY FOR EP 3
   TRAIN ACCURACY :  85.2 %
   TEST ACCURACY :  76.3 %
ACCURACY FOR EP 4
   TRAIN ACCURACY :  91.12 %
   TEST ACCURACY :  81.39999999999999 %
ACCURACY FOR EP 5
   TRAIN ACCURACY :  84.06 %
   TEST ACCURACY :  74.6 %


## Section C

We would choose Perceptron over Naive Bayes because it gave us higher accuracy (except for some epochs runs).

The accuracy we got from each model on the test set is pretty close.
    
Perceptron is better due to how the model learns data, Perceptron makes an initial assumption and then improves (learns from its mistakes) via doing multiple passes on the dataset